Antes de executar este notebook, configure um ambiente virtual e execute os comandos a seguir a partir da raíz do projeto:
```shell
$ pip install -r requirements.txt
$ pip install -e .
```

O local deste notebook é `notebooks/examples/extract_text_with_apache_tika.ipynb`.

In [1]:
pip install pandas termcolor

Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
import os
import re
from pathlib import Path

import requests
import pandas as pd
from termcolor import colored

In [3]:
ROOT = Path(os.path.abspath('.')).parent
CNPJS_DATASET = ROOT / "data" / "cnpjs.json"

In [4]:
cnpjs_encontrados = pd.read_json(CNPJS_DATASET)
cnpjs_encontrados

,cnpj,qtd_encontrados,municipio,data_publicacao,nome_arquivo,caminho_arquivo
0,02.361.033/0001-92,1,4216602,2020-03-09,59f04d547cad602737ae9a24c95246db05bfd975,data-processed/4216602/2020-03-09/59f04d547cad...
1,05.691.145/0001-08,1,4216602,2020-03-09,59f04d547cad602737ae9a24c95246db05bfd975,data-processed/4216602/2020-03-09/59f04d547cad...
2,82.892.274/0001-05,1,4216602,2020-08-28,ffa6d342b2510ee4b86613d31d9c1ed315f51134,data-processed/4216602/2020-08-28/ffa6d342b251...
3,10.425.359/0001-64,1,4216602,2020-05-14,88bdc47427b36317f65293cd7bdc74294c283c20,data-processed/4216602/2020-05-14/88bdc47427b3...
4,02.299.772/0001-00,1,4216602,2020-12-03,5ef7b675e50bd17a7f33284302b4ad34b9959ec1,data-processed/4216602/2020-12-03/5ef7b675e50b...
...,...,...,...,...,...,...
2033,24.419.445/0001-79,1,4202404,2020-05-28,8e840b9d68c2f6b8c28d7f36136f557b42516868,data-processed/4202404/2020-05-28/8e840b9d68c2...
2034,08.862.530/0001-50,1,4202404,2020-05-28,8e840b9d68c2f6b8c28d7f36136f557b42516868,data-processed/4202404/2020-05-28/8e840b9d68c2...
2035,82.662.958/0001-02,2,4202404,2020-05-06,b134c23a90453a384a1e796388714d7528d2b2d3,data-processed/4202404/2020-05-06/b134c23a9045...
2036,92.959.006/0001-09,1,4202404,2020-05-06,b134c23a90453a384a1e796388714d7528d2b2d3,data-processed/4202404/2020-05-06/b134c23a9045...


In [5]:
cnpjs_distintos = cnpjs_encontrados.groupby('cnpj', as_index=False).sum().drop('municipio', axis=1)
cnpjs_distintos

,cnpj,qtd_encontrados
0,00.000.028/0001-29,1
1,00.065.973/0001-09,1
2,00.070.408/0001-30,2
3,00.110.139/0001-98,1
4,00.129.000/0001-96,3
...,...,...
1122,95.792.248/0001-49,1
1123,96.730.809/0001-48,1
1124,97.533.241/0001-38,1
1125,97.542.342/0001-75,1


In [6]:
cnpjs_distintos = cnpjs_distintos.rename(columns={'cnpj': 'cnpj_original'})
cnpjs_distintos['cnpj'] = cnpjs_distintos['cnpj_original'].str.replace(r'[./-]', '', regex=True)
cnpjs_distintos = cnpjs_distintos[['cnpj', 'cnpj_original', 'qtd_encontrados']]
cnpjs_distintos

,cnpj,cnpj_original,qtd_encontrados
0,00000028000129,00.000.028/0001-29,1
1,00065973000109,00.065.973/0001-09,1
2,00070408000130,00.070.408/0001-30,2
3,00110139000198,00.110.139/0001-98,1
4,00129000000196,00.129.000/0001-96,3
...,...,...,...
1122,95792248000149,95.792.248/0001-49,1
1123,96730809000148,96.730.809/0001-48,1
1124,97533241000138,97.533.241/0001-38,1
1125,97542342000175,97.542.342/0001-75,1


In [7]:
def infos_receita(cnpj):
    dados_cnpj = json.loads(requests.get(f"https://minhareceita.org/{cnpj}").text)
    return dados_cnpj

In [8]:
infos_receita_cnpjs = [infos_receita(cnpj) for cnpj in cnpjs_distintos['cnpj']]

In [9]:
infos_cnpjs = cnpjs_distintos.merge(pd.DataFrame(infos_receita_cnpjs), on='cnpj')
infos_cnpjs

,cnpj,cnpj_original,qtd_encontrados,identificador_matriz_filial,descricao_matriz_filial,razao_social,nome_fantasia,situacao_cadastral,descricao_situacao_cadastral,data_situacao_cadastral,...,porte,descricao_porte,opcao_pelo_simples,data_opcao_pelo_simples,data_exclusao_do_simples,opcao_pelo_mei,situacao_especial,data_situacao_especial,qsa,cnaes_secundarias
0,00000028000129,00.000.028/0001-29,1,1,Matriz,TARGET ENGENHARIA E CONSULTORIA LTDA.,,2,Ativa,2005-03-12,...,5,Demais,False,None,None,False,,None,"[{'cnpj': '00000028000129', 'identificador_de_...","[{'codigo': 6204000, 'descricao': 'Consultoria..."
1,00065973000109,00.065.973/0001-09,1,1,Matriz,SOCIEDADE BENEFICENTE ARNOLD HADLICH,,2,Ativa,2005-08-27,...,5,Demais,False,None,None,False,,None,"[{'cnpj': '00065973000109', 'identificador_de_...","[{'codigo': 8511200, 'descricao': 'Educação in..."
2,00070408000130,00.070.408/0001-30,2,1,Matriz,VIACAO NOSSA SENHORA DOS NAVEGANTES LTDA,,2,Ativa,2005-11-03,...,5,Demais,False,None,None,False,,None,"[{'cnpj': '00070408000130', 'identificador_de_...","[{'codigo': 7711000, 'descricao': 'Locação de ..."
3,00110139000198,00.110.139/0001-98,1,1,Matriz,EDITORA JORNALISTICA LANZINI LTDA,,8,Baixada,2008-12-31,...,5,Demais,False,None,None,False,,None,None,None
4,00129000000196,00.129.000/0001-96,3,1,Matriz,CENTRO DE REABILITACAO JOVENS LIVRES,,2,Ativa,2005-11-03,...,5,Demais,False,None,None,False,,None,"[{'cnpj': '00129000000196', 'identificador_de_...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1122,95792248000149,95.792.248/0001-49,1,1,Matriz,CONSTRUTORA E INCORPORADORA SANTOLIN LTDA,,2,Ativa,2004-10-30,...,1,Microempresa,False,None,None,False,,None,"[{'cnpj': '95792248000149', 'identificador_de_...","[{'codigo': 4110700, 'descricao': 'Incorporaçã..."
1123,96730809000148,96.730.809/0001-48,1,1,Matriz,C P R COMERCIAL DE PRODUTOS RADIOLOGICOS LTDA,C P R,2,Ativa,2002-08-24,...,5,Demais,False,None,None,False,,None,"[{'cnpj': '96730809000148', 'identificador_de_...","[{'codigo': 7739002, 'descricao': 'Aluguel de ..."
1124,97533241000138,97.533.241/0001-38,1,1,Matriz,M.H.M. DO COUTO - COMERCIAL,ROYAL SHOPPING DA SAUDE,2,Ativa,2011-07-07,...,3,Empresa de pequeno porte,True,2019-01-01,None,False,,None,None,"[{'codigo': 4751201, 'descricao': 'Comércio va..."
1125,97542342000175,97.542.342/0001-75,1,1,Matriz,RT IMOBILIARIA LTDA,,2,Ativa,2011-07-11,...,1,Microempresa,True,2015-01-01,None,False,,None,"[{'cnpj': '97542342000175', 'identificador_de_...","[{'codigo': 6821802, 'descricao': 'Corretagem ..."


In [85]:
# pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)  
infos_cnpjs.sort_values(by='qtd_encontrados', ascending=False).iloc[0:20]

,cnpj,cnpj_original,qtd_encontrados,identificador_matriz_filial,descricao_matriz_filial,razao_social,nome_fantasia,situacao_cadastral,descricao_situacao_cadastral,data_situacao_cadastral,motivo_situacao_cadastral,nome_cidade_exterior,codigo_natureza_juridica,data_inicio_atividade,cnae_fiscal,cnae_fiscal_descricao,descricao_tipo_logradouro,logradouro,numero,complemento,bairro,cep,uf,codigo_municipio,municipio,ddd_telefone_1,ddd_telefone_2,ddd_fax,qualificacao_do_responsavel,capital_social,porte,descricao_porte,opcao_pelo_simples,data_opcao_pelo_simples,data_exclusao_do_simples,opcao_pelo_mei,situacao_especial,data_situacao_especial,qsa,cnaes_secundarias
1083,83021808000182,83.021.808/0001-82,351,1,Matriz,MUNICIPIO DE CHAPECO,PREFEITURA MUNICIPAL DE CHAPECO,2,Ativa,2005-11-03,0,,1244,1974-12-23,8411600,Administração pública em geral,AVENIDA,GETULIO DORNELES VARGAS - S 957,957,LETRA S,CENTRO,89812900,SC,8081,CHAPECO,49 33218474,,,5,0.0,5,Demais,False,None,None,False,,None,None,None
1076,82662958000102,82.662.958/0001-02,317,1,Matriz,FUNDACAO UNIVERSIDADE REGIONAL DE BLUMENAU - FURB,,2,Ativa,2001-01-06,0,,1120,1970-02-19,8532500,Educação superior - graduação e pós-graduação,RUA,ANTONIO DA VEIGA,140,,ITOUPAVA SECA,89030903,SC,8047,BLUMENAU,,,,5,0.0,5,Demais,False,None,None,False,,None,None,"[{'codigo': 8630504, 'descricao': 'Atividade odontológica'}, {'codigo': 8640202, 'descricao': 'Laboratórios clínicos'}, {'codigo': 8640214, 'descricao': 'Serviços de bancos de células e tecidos humanos'}, {'codigo': 8650004, 'descricao': 'Atividades de fisioterapia'}, {'codigo': 9001901, 'descricao': 'Produção teatral'}, {'codigo': 9001902, 'descricao': 'Produção musical'}, {'codigo': 9001903, 'descricao': 'Produção de espetáculos de dança'}, {'codigo': 9001904, 'descricao': 'Produção de espetáculos circenses, de marionetes e similares'}, {'codigo': 9001906, 'descricao': 'Atividades de sonorização e de iluminação'}, {'codigo': 9001999, 'descricao': 'Artes cênicas, espetáculos e atividades complementares não especificados anteriormente'}, {'codigo': 8593700, 'descricao': 'Ensino de idiomas'}, {'codigo': 8640201, 'descricao': 'Laboratórios de anatomia patológica e citológica'}, {'codigo': 6021700, 'descricao': 'Atividades de televisão aberta'}, {'codigo': 6022502, 'descricao': 'Atividades relacionadas à televisão por assinatura, exceto programadoras'}, {'codigo': 6010100, 'descricao': 'Atividades de rádio'}, {'codigo': 5811500, 'descricao': 'Edição de livros'}, {'codigo': 5813100, 'descricao': 'Edição de revistas'}, {'codigo': 5823900, 'descricao': 'Edição integrada à impressão de revistas'}, {'codigo': 5821200, 'descricao': 'Edição integrada à impressão de livros'}, {'codigo': 8219999, 'descricao': 'Preparação de documentos e serviços especializados de apoio administrativo não especificados anteriormente'}, {'codigo': 8520100, 'descricao': 'Ensino médio'}, {'codigo': 8610101, 'descricao': 'Atividades de atendimento hospitalar, exceto pronto-socorro e unidades para atendimento a urgências'}, {'codigo': 8630503, 'descricao': 'Atividade médica ambulatorial restrita a consultas'}]"
751,27174093001875,27.174.093/0018-75,80,2,Filial,MUNICIPIO DA SERRA,SECRETARIA MUNICIPAL DE OBRAS,2,Ativa,2001-01-04,0,,1244,2001-01-04,8411600,Administração pública em geral,RUA,MAESTRO ANTONIO CICERO,111,ANEXO: 1 ANDAR;,CACAROCA,29176439,ES,5699,SERRA,27 32912119,27 32912121,27 32912112,5,0.0,5,Demais,False,None,None,False,,None,None,None
77,02361033000192,02.361.033/0001-92,51,1,Matriz,FUNDACAO MUNICIPAL DE ESPORTES DE SAO JOSE,FUNESJ,2,Ativa,2018-08-29,0,,1155,1995-03-22,8412400,"Regulação das atividades de saúde, educação, serviços culturais e outros serviços sociais",RUA,DOMINGOS ANDRE ZANINI,300,PROXIMO AO FORUM,CAMPINAS,88117200,SC,8327,SAO JOSE,48 3810062,,48 3810076,16,0.0,5,Demais,False,None,None,False,,None,None,None
233,06320870000124,06.320.870/0001-24,38,1,Matriz,FUNDACAO MUNICIPAL DE CULTURA E TURISMO DE SAO JOSE,,2,Ativa,2004-01-06,0,,1155,2004-01-06,8412

In [96]:
mais_encontrados = infos_cnpjs.sort_values(by='qtd_encontrados', ascending=False)
escolhido = mais_encontrados.iloc[3]
escolhido

cnpj                                                                                                       02361033000192
cnpj_original                                                                                          02.361.033/0001-92
qtd_encontrados                                                                                                        51
identificador_matriz_filial                                                                                             1
descricao_matriz_filial                                                                                            Matriz
razao_social                                                                   FUNDACAO MUNICIPAL DE ESPORTES DE SAO JOSE
nome_fantasia                                                                                                      FUNESJ
situacao_cadastral                                                                                                      2
descricao_situacao_cadas

In [97]:
ocorrencias_escolhido = cnpjs_encontrados[cnpjs_encontrados['cnpj'] == escolhido['cnpj_original']].sort_values(by='data_publicacao')
ocorrencias_escolhido

,cnpj,qtd_encontrados,municipio,data_publicacao,nome_arquivo,caminho_arquivo
14,02.361.033/0001-92,1,4216602,2020-01-02,e56caa6d6a50a0d65dd03d80d195b4aa1c50d57f,data-processed/4216602/2020-01-02/e56caa6d6a50a0d65dd03d80d195b4aa1c50d57f/content.txt
15,02.361.033/0001-92,1,4216602,2020-01-02,90610e967b1e80acbafbe29236f23313f921aaac,data-processed/4216602/2020-01-02/90610e967b1e80acbafbe29236f23313f921aaac/content.txt
16,02.361.033/0001-92,1,4216602,2020-01-02,c33fe2f93220519a3a600c64fd88f33f8cddec31,data-processed/4216602/2020-01-02/c33fe2f93220519a3a600c64fd88f33f8cddec31/content.txt
17,02.361.033/0001-92,14,4216602,2020-02-28,0dcffc55461d113380d596ff6cd857e3b25f4a1e,data-processed/4216602/2020-02-28/0dcffc55461d113380d596ff6cd857e3b25f4a1e/content.txt
28,02.361.033/0001-92,28,4216602,2020-02-28,f6ed15b3345e07d0573290d9c33b6a05203da6a1,data-processed/4216602/2020-02-28/f6ed15b3345e07d0573290d9c33b6a05203da6a1/content.txt
11,02.361.033/0001-92,1,4216602,2020-03-03,a9e0d71ecdfbf5c4d1688cbfa701a50497c16939,data-processed/4216602/2020-03-03/a9e0d71ecdfbf5c4d1688cbfa701a50497c16939/content.txt
81,02.361.033/0001-92,1,4216602,2020-03-06,01350a73bad6650cdb7c1d476dfb28adb718c070,data-processed/4216602/2020-03-06/01350a73bad6650cdb7c1d476dfb28adb718c070/content.txt
0,02.361.033/0001-92,1,4216602,2020-03-09,59f04d547cad602737ae9a24c95246db05bfd975,data-processed/4216602/2020-03-09/59f04d547cad602737ae9a24c95246db05bfd975/content.txt
51,02.361.033/0001-92,1,4216602,2020-03-12,1ce0e2045a9430dc42c61883a24f3a34a40f15b2,data-processed/4216602/2020-03-12/1ce0e2045a9430dc42c61883a24f3a34a40f15b2/content.txt
53,02.361.033/0001-92,1,4216602,2020-03-12,93259de1521121eadf8a91afecd9a965e56ed8f2,data-processed/4216602/2020-03-12/93259de1521121eadf8a91afecd9a965e56ed8f2/content.txt


In [98]:
ocorrencia_template = \
"""{divisor}
{titulo}
{subdivisor}
{metadados}
{divisor}

{texto}
"""

def imprime_ocorrencias(cnpj, titulo):
    arquivos_distintos = cnpjs_encontrados[cnpjs_encontrados['cnpj'] == cnpj].sort_values(by='data_publicacao')
    cnpj_escape = re.escape(cnpj)
    for _, ocorrencias in arquivos_distintos.iterrows():
        arquivo = ROOT / ocorrencias['caminho_arquivo']
        texto = arquivo.read_text()
        ocorrencias_com_contextos = re.findall(fr'.{{0,1000}}{cnpj_escape}.{{0,1000}}', texto, re.DOTALL)
        for contexto in ocorrencias_com_contextos:
            cnpj_em_destaque = contexto.replace(cnpj, colored(cnpj, 'grey', 'on_yellow'))
            metadados = "\n".join([
                f"Código do município da publicação: {ocorrencias['municipio']}",
                f"Data de publicação: {ocorrencias['data_publicacao']}",
            ])
            print(arquivo)
            print(ocorrencia_template.format(
                divisor="="*100,
                subdivisor="-"*100,
                titulo=titulo,
                metadados=metadados,
                texto=cnpj_em_destaque,
            ))

In [99]:
titulo = f"{escolhido['razao_social']} ({escolhido['cnpj_original']})"
imprime_ocorrencias(cnpj=escolhido['cnpj_original'], titulo=titulo)

/home/gcc/work/projects/querido-diario/toolbox/data-processed/4216602/2020-01-02/e56caa6d6a50a0d65dd03d80d195b4aa1c50d57f/content.txt
FUNDACAO MUNICIPAL DE ESPORTES DE SAO JOSE (02.361.033/0001-92)
----------------------------------------------------------------------------------------------------
Código do município da publicação: 4216602
Data de publicação: 2020-01-02



EDITAL DE SELEÇÃO DE PROJETOS DE EVENTOS ESPORTIVOS Nº 001/FUNESJ/2020


O MUNICÍPIO DE SÃO JOSÉ, por intermédio da FUNDAÇÃO MUNICIPAL DE ESPORTES E LAZER, com sede à Avenida Acioni de Souza Filho, s/nº. 403, CEP. 88.101-175, São José (SC), inscrita no CNPJ sob nº. 02.361.033/0001-92, com fulcro na Lei nº 13.019, de 31 de julho de 2014 e no Decreto nº 8.623, de 31 de agosto de 2017, torna público o presente EDITAL DE CHAMAMENTO PÚBLICO visando à seleção de organizações da sociedade civil interessadas em celebrar TERMO DE FOMENTO com o objetivo de executar PROJETOS DE EVENTOS ESPORTIVOS – no Município de São José (SC)

In [18]:
mais_encontrados.columns

Index(['cnpj', 'cnpj_original', 'qtd_encontrados',
       'identificador_matriz_filial', 'descricao_matriz_filial',
       'razao_social', 'nome_fantasia', 'situacao_cadastral',
       'descricao_situacao_cadastral', 'data_situacao_cadastral',
       'motivo_situacao_cadastral', 'nome_cidade_exterior',
       'codigo_natureza_juridica', 'data_inicio_atividade', 'cnae_fiscal',
       'cnae_fiscal_descricao', 'descricao_tipo_logradouro', 'logradouro',
       'numero', 'complemento', 'bairro', 'cep', 'uf', 'codigo_municipio',
       'municipio', 'ddd_telefone_1', 'ddd_telefone_2', 'ddd_fax',
       'qualificacao_do_responsavel', 'capital_social', 'porte',
       'descricao_porte', 'opcao_pelo_simples', 'data_opcao_pelo_simples',
       'data_exclusao_do_simples', 'opcao_pelo_mei', 'situacao_especial',
       'data_situacao_especial', 'qsa', 'cnaes_secundarias'],
      dtype='object')

In [86]:
mais_encontrados.groupby('cnae_fiscal_descricao')[['cnae_fiscal_descricao', 'qtd_encontrados']].sum().sort_values('qtd_encontrados', ascending=False).head()

,qtd_encontrados
cnae_fiscal_descricao,
Administração pública em geral,533
Educação superior - graduação e pós-graduação,317
Comércio atacadista de medicamentos e drogas de uso humano,164
Atividades de associações de defesa de direitos sociais,111
"Regulação das atividades de saúde, educação, serviços culturais e outros serviços sociais",96


In [82]:
mais_encontrados.groupby('cnae_fiscal_descricao')[['cnae_fiscal_descricao', 'qtd_encontrados']].mean().sort_values('qtd_encontrados', ascending=False).head()

,qtd_encontrados
cnae_fiscal_descricao,
Educação superior - graduação e pós-graduação,317.000000
Administração pública em geral,28.052632
"Regulação das atividades de saúde, educação, serviços culturais e outros serviços sociais",16.000000
"Emissão de vales-alimentação, vales-transporte e similares",7.000000
Atividades de sonorização e de iluminação,6.800000


In [40]:
associacoes_sociais = mais_encontrados[mais_encontrados['cnae_fiscal_descricao'] == 'Atividades de associações de defesa de direitos sociais'].sort_values('qtd_encontrados', ascending=False).head()
associacoes_sociais

,cnpj,cnpj_original,qtd_encontrados,identificador_matriz_filial,descricao_matriz_filial,razao_social,nome_fantasia,situacao_cadastral,descricao_situacao_cadastral,data_situacao_cadastral,motivo_situacao_cadastral,nome_cidade_exterior,codigo_natureza_juridica,data_inicio_atividade,cnae_fiscal,cnae_fiscal_descricao,descricao_tipo_logradouro,logradouro,numero,complemento,bairro,cep,uf,codigo_municipio,municipio,ddd_telefone_1,ddd_telefone_2,ddd_fax,qualificacao_do_responsavel,capital_social,porte,descricao_porte,opcao_pelo_simples,data_opcao_pelo_simples,data_exclusao_do_simples,opcao_pelo_mei,situacao_especial,data_situacao_especial,qsa,cnaes_secundarias
75,02325057000196,02.325.057/0001-96,8,1,Matriz,APAE-ASSOCIACAO DE PAIS E AMIGOS DOS EXCEPCION...,,2,Ativa,2005-11-03,0,,3999,1997-09-09,9430800,Atividades de associações de defesa de direito...,RUA,VEREADOR JORGE SIMOES,10,,ITAPEBUSSU,29210155,ES,5647,GUARAPARI,27 32622209,,,16,0.0,5,Demais,False,None,None,False,,None,"[{'cnpj': '02325057000196', 'identificador_de_...","[{'codigo': 9493600, 'descricao': 'Atividades ..."
248,07098848000144,07.098.848/0001-44,7,1,Matriz,ASSOCIACAO PEDAGOGICA EURIPEDES BARSANULFO,,2,Ativa,2004-10-06,0,,3999,2004-10-06,9430800,Atividades de associações de defesa de direito...,RUA,ANITAPOLIS,85,,VORSTARDT,89015350,SC,8047,BLUMENAU,47 33231748,,,16,0.0,5,Demais,False,None,None,False,,None,"[{'cnpj': '07098848000144', 'identificador_de_...","[{'codigo': 8800600, 'descricao': 'Serviços de..."
1040,79371803000102,79.371.803/0001-02,5,1,Matriz,ASSOC BLUMEN DE AMIGOS DOS DEFICIENTES AUDITIVOS,ABADA,2,Ativa,2005-11-03,0,,3999,1989-11-20,9430800,Atividades de associações de defesa de direito...,RUA,MAX HERING,123,,VICTOR KONDER,89012510,SC,8047,BLUMENAU,47 3402463,,,16,0.0,5,Demais,False,None,None,False,,None,"[{'cnpj': '79371803000102', 'identificador_de_...","[{'codigo': 9493600, 'descricao': 'Atividades ..."
1073,82656554000106,82.656.554/0001-06,4,1,Matriz,ASSOC DE PAIS E AMIGOS DOS EXCEPCIONAIS DE BLU...,APAE,2,Ativa,1998-10-27,0,,3999,1969-12-31,9430800,Atividades de associações de defesa de direito...,RUA,CASIMIRO DE ABREU,216,,VILA NOVA,89035600,SC,8047,BLUMENAU,,,,16,0.0,5,Demais,False,None,None,False,,None,"[{'cnpj': '82656554000106', 'identificador_de_...","[{'codigo': 1091102, 'descricao': 'Fabricação ..."
17,00612230000101,00.612.230/0001-01,4,1,Matriz,ASSOCIACAO LAR SEMENTE DO AMOR,LAR SEMENTE DO AAMOR,2,Ativa,1998-07-28,0,,3999,1995-03-22,9430800,Atividades de associações de defesa de direito...,AVENIDA,AUGUSTO RUSCHI,1541,,BALNEARIO DE CARAPEBUS,29164830,ES,5699,SERRA,27 34216190,,27 33471550,16,0.0,5,Demais,False,None,None,False,,None,"[{'cnpj': '00612230000101', 'identificador_de_...","[{'codigo': 9493600, 'descricao': 'Atividades ..."


In [50]:

    
for _, associacao in associacoes_sociais.iterrows():
    titulo = f"{associacao['razao_social']} ({associacao['cnpj_original']})"
    imprime_ocorrencias(cnpj=associacao['cnpj_original'], titulo=titulo)

APAE-ASSOCIACAO DE PAIS E AMIGOS DOS EXCEPCIONAIS DE GUARAPARI (02.325.057/0001-96)
Código do município da publicação: 3202405
Data de publicação: 2020-07-06




MUNICIPIO DE GUARAPARI

ESTADO DO ESPIRITO SANTO
GABINETE DO PREFEITO


LEI N°. 4428/2020
DISPÕE SOBRE TERMO DE FOMENTO  COM ENTIDADE SEM FINS LUCRATIVOS E DÁ OUTRAS PROVIDÊNCIAS.
O PREFEITO MUNICIPAL DE GUARAPARI, Estado do Espírito Santo, no uso de suas atribuições legais, alicerçado nas disposições do art. 88, Inciso V, da Lei Orgânica do Município - LOM, faz saber que a Câmara Municipal APROVOU e ele SANCIONA a seguinte

LEI:

Art. 1º - Fica o Poder Executivo autorizado a firmar Termo de Fomento, no valor total R$ 50.000,00 (cinquenta mil reais), como forma de subvenção social, dentro de rubricas da Secretaria Municipal do Trabalho, Assistência e Cidadania – SETAC, no programa orçamentário municipal para o exercício financeiro de 2020, com a ASSOCIAÇÃO DE PAIS E AMIGOS DOS EXCEPCIONAIS DE GUARAPARI / CENTRO DE ATENDIMENTO 

ASSOCIACAO LAR SEMENTE DO AMOR (00.612.230/0001-01)
Código do município da publicação: 3205002
Data de publicação: 2020-12-14

a- CONCASE, no uso de suas atribuições legais, conforme a lei 3898/2012.

Considerando a Resolução 046/2019 de 18/10/2019, artigos 13º incisos I e II.

Considerando a Resolução 001/2020 que dispõe sobre o registro de entidade para 2 anos e certificados emitidos para 4 anos.

Considerando a aprovação de registro de entidades nas plenárias do CONCASE realizadas nas datas 16/10/2019, 06/11/2019 e 11/12/2019.

Considerando a retomada das atividades administrativas, e que identificou a necessidade de emissão/correção nos documentos de registros concedidos.

Considerando deliberação em plenária do CONCASE de 09/12/2020.

RESOLVE:

Art. 1º Alterar a Resolução 001/2020 - CONCASE  no que se refere ao prazo de validade do certificado de registro das organizações da sociedade civil para 4 anos a contar da data de sua aprovação na plenária de 11/12/2019, conforme lista aba

In [35]:
mais_encontrados[mais_encontrados['qtd_encontrados'] > 10].groupby('cnae_fiscal_descricao')[['cnae_fiscal_descricao', 'qtd_encontrados']].sum().sort_values('qtd_encontrados', ascending=False).head()

,qtd_encontrados
cnae_fiscal_descricao,
Administração pública em geral,500
Educação superior - graduação e pós-graduação,317
"Regulação das atividades de saúde, educação, serviços culturais e outros serviços sociais",89
Comércio atacadista de medicamentos e drogas de uso humano,54
Atividades de sonorização e de iluminação,27
